# Feature Engineering

In [ ]:
# Take the first n numbers from the upc as a string 
# Take the upc length as a string
#o# Make departments all separate features and count the number of items purchased under each department
# Make broader departments such as Clothing for all Clothing
#o# Weekend vs Weekday indicator (1 and 0)
# Make null a binary indicator (1 and 0)
# Replace upc null with "no_upc"
# Replace finelinenumber null with "no_fineline"
#o# Ratio of total_scancount / num_unique_upc = avg_scancount_per_upc
#o# Ratio of total_scancount / num_unique_dept = avg_scancount_per_dept
# Ratio of num_unique_fineline / num_unique_dept = fineline_to_dept_ratio
# Ratio of num_unique_upc / num_unique_fineline = avg_upc_per_fineline (maybe)
# Ratio of num_unique_upc / num_unique_dept = avg_upc_per_dept

In [3]:
# Imports
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import set_config
set_config(transform_output='pandas')

In [4]:
# Import data
df = pd.read_csv('../data-ignore/train.csv', dtype={'Upc': str})
df.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000.0
1,30,7,Friday,60538815980,1,SHOES,8931.0
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504.0
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565.0
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017.0


In [17]:
# Create a new feature indicating if scancount is negative
df['return'] = df['ScanCount'].apply(lambda x: 1 if x < 0 else 0)
df

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,return
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000.0,1
1,30,7,Friday,60538815980,1,SHOES,8931.0,0
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504.0,0
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565.0,0
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017.0,0
...,...,...,...,...,...,...,...,...
647049,39,191346,Sunday,32390001778,1,PHARMACY OTC,1118.0,0
647050,39,191346,Sunday,7874205336,1,FROZEN FOODS,1752.0,0
647051,39,191346,Sunday,4072,1,PRODUCE,4170.0,0
647052,8,191347,Sunday,4190007664,1,DAIRY,1512.0,0


In [18]:
# Group by VisitNumber to get features and target (triptype)
df_groupby_visitnumber = df.groupby('VisitNumber').agg(triptype=('TripType','first'),
                              weekday=('Weekday','first'),
                              num_unique_upc=('Upc','nunique'),
                              avg_scancount=('ScanCount','mean'),
                              total_scancount=('ScanCount','sum'),
                              num_unique_dept=('DepartmentDescription','nunique'),
                              num_unique_fileline=('FinelineNumber','nunique'),
                              contains_return=('return','max')
                              )
df_groupby_visitnumber.reset_index(inplace=True)
df_groupby_visitnumber

,VisitNumber,triptype,weekday,num_unique_upc,avg_scancount,total_scancount,num_unique_dept,num_unique_fileline,contains_return
0,5,999,Friday,1,-1.000000,-1,1,1,1
1,7,30,Friday,2,1.000000,2,2,2,0
2,8,26,Friday,20,1.217391,28,6,16,1
3,9,8,Friday,3,1.000000,3,2,3,0
4,10,8,Friday,3,1.000000,3,2,3,0
...,...,...,...,...,...,...,...,...,...
95669,191343,25,Sunday,7,1.285714,9,3,5,0
95670,191344,22,Sunday,5,1.000000,5,2,3,0
95671,191345,39,Sunday,13,1.307692,17,8,12,0
95672,191346,39,Sunday,17,1.000000,17,8,16,0


### OneHotEncode the Department and then Groupby to get the ScanCount per Dept per VisitNumber

In [6]:
# OHE the Department
transformer = ColumnTransformer([
  ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['DepartmentDescription'])
], remainder='passthrough', verbose_feature_names_out=False)

df_ohe_dept = transformer.fit_transform(df)
df_ohe_dept

,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,DepartmentDescription_AUTOMOTIVE,DepartmentDescription_BAKERY,DepartmentDescription_BATH AND SHOWER,DepartmentDescription_BEAUTY,DepartmentDescription_BEDDING,DepartmentDescription_BOOKS AND MAGAZINES,DepartmentDescription_BOYS WEAR,DepartmentDescription_BRAS & SHAPEWEAR,...,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,DepartmentDescription_nan,TripType,VisitNumber,Weekday,Upc,ScanCount,FinelineNumber
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,999,5,Friday,68113152929,-1,1000.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,30,7,Friday,60538815980,1,8931.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,30,7,Friday,7410811099,1,4504.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,26,8,Friday,2238403510,2,3565.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,26,8,Friday,2006613744,2,1017.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,39,191346,Sunday,32390001778,1,1118.0
647050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,39,191346,Sunday,7874205336,1,1752.0
647051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,39,191346,Sunday,4072,1,4170.0
647052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8,191347,Sunday,4190007664,1,1512.0


In [7]:
# Multiply all ohe department by the ScanCount
ohe_features = [feature for feature in df_ohe_dept.columns.to_list() if 'DepartmentDescription_' in feature]

new_df = pd.DataFrame(df_ohe_dept['VisitNumber'])
for feature in ohe_features:
  new_df[feature] = df_ohe_dept[feature] * df_ohe_dept['ScanCount']

In [8]:
new_df

,VisitNumber,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,DepartmentDescription_AUTOMOTIVE,DepartmentDescription_BAKERY,DepartmentDescription_BATH AND SHOWER,DepartmentDescription_BEAUTY,DepartmentDescription_BEDDING,DepartmentDescription_BOOKS AND MAGAZINES,DepartmentDescription_BOYS WEAR,...,DepartmentDescription_SEASONAL,DepartmentDescription_SERVICE DELI,DepartmentDescription_SHEER HOSIERY,DepartmentDescription_SHOES,DepartmentDescription_SLEEPWEAR/FOUNDATIONS,DepartmentDescription_SPORTING GOODS,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,DepartmentDescription_nan
0,5,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
1,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647049,191346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
647050,191346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
647051,191346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
647052,191347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Groupby OHE dept by VisitNumber, totaling the scancount per department
new_df_groupby = new_df.groupby('VisitNumber').sum()
new_df_groupby

,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,DepartmentDescription_AUTOMOTIVE,DepartmentDescription_BAKERY,DepartmentDescription_BATH AND SHOWER,DepartmentDescription_BEAUTY,DepartmentDescription_BEDDING,DepartmentDescription_BOOKS AND MAGAZINES,DepartmentDescription_BOYS WEAR,DepartmentDescription_BRAS & SHAPEWEAR,...,DepartmentDescription_SEASONAL,DepartmentDescription_SERVICE DELI,DepartmentDescription_SHEER HOSIERY,DepartmentDescription_SHOES,DepartmentDescription_SLEEPWEAR/FOUNDATIONS,DepartmentDescription_SPORTING GOODS,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,DepartmentDescription_nan
VisitNumber,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
191344,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
191345,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
new_df_groupby.iloc[13].head(60)

DepartmentDescription_1-HR PHOTO                     0.0
DepartmentDescription_ACCESSORIES                    0.0
DepartmentDescription_AUTOMOTIVE                     0.0
DepartmentDescription_BAKERY                         0.0
DepartmentDescription_BATH AND SHOWER                0.0
DepartmentDescription_BEAUTY                         0.0
DepartmentDescription_BEDDING                        0.0
DepartmentDescription_BOOKS AND MAGAZINES            0.0
DepartmentDescription_BOYS WEAR                      0.0
DepartmentDescription_BRAS & SHAPEWEAR               0.0
DepartmentDescription_CAMERAS AND SUPPLIES           0.0
DepartmentDescription_CANDY, TOBACCO, COOKIES        0.0
DepartmentDescription_CELEBRATION                    0.0
DepartmentDescription_COMM BREAD                     0.0
DepartmentDescription_CONCEPT STORES                 0.0
DepartmentDescription_COOK AND DINE                  2.0
DepartmentDescription_DAIRY                          6.0
DepartmentDescription_DSD GROCE

In [11]:
# Ratio of total_scancount / num_unique_upc = avg_scancount_per_upc
# Ratio of total_scancount / num_unique_dept = avg_scancount_per_dept

In [12]:
df_groupby_visitnumber

,VisitNumber,triptype,weekday,num_unique_upc,avg_scancount,total_scancount,num_unique_dept,num_unique_fileline
0,5,999,Friday,1,-1.000000,-1,1,1
1,7,30,Friday,2,1.000000,2,2,2
2,8,26,Friday,20,1.217391,28,6,16
3,9,8,Friday,3,1.000000,3,2,3
4,10,8,Friday,3,1.000000,3,2,3
...,...,...,...,...,...,...,...,...
95669,191343,25,Sunday,7,1.285714,9,3,5
95670,191344,22,Sunday,5,1.000000,5,2,3
95671,191345,39,Sunday,13,1.307692,17,8,12
95672,191346,39,Sunday,17,1.000000,17,8,16


In [ ]:
df_groupby_visitnumber['avg_scancount_per_upc'] = df_groupby_visitnumber['total_scancount'] / (df_groupby_visitnumber['num_unique_upc'] + 1)
df_groupby_visitnumber['avg_scancount_per_dept'] = df_groupby_visitnumber['total_scancount'] / (df_groupby_visitnumber['num_unique_dept'] + 1)

df_groupby_visitnumber

,VisitNumber,triptype,weekday,num_unique_upc,avg_scancount,total_scancount,num_unique_dept,num_unique_fileline,avg_scancount_per_upc,avg_scancount_per_dept
0,5,999,Friday,1,-1.000000,-1,1,1,-1.000000,-1.000000
1,7,30,Friday,2,1.000000,2,2,2,1.000000,1.000000
2,8,26,Friday,20,1.217391,28,6,16,1.400000,4.666667
3,9,8,Friday,3,1.000000,3,2,3,1.000000,1.500000
4,10,8,Friday,3,1.000000,3,2,3,1.000000,1.500000
...,...,...,...,...,...,...,...,...,...,...
95669,191343,25,Sunday,7,1.285714,9,3,5,1.285714,3.000000
95670,191344,22,Sunday,5,1.000000,5,2,3,1.000000,2.500000
95671,191345,39,Sunday,13,1.307692,17,8,12,1.307692,2.125000
95672,191346,39,Sunday,17,1.000000,17,8,16,1.000000,2.125000


In [ ]:
df_groupby_visitnumber['weekend'] = df_groupby_visitnumber['weekday'].apply(lambda x: 1 if x in ['Saturday','Sunday'] else 0)
df_groupby_visitnumber

,VisitNumber,triptype,weekday,num_unique_upc,avg_scancount,total_scancount,num_unique_dept,num_unique_fileline,avg_scancount_per_upc,avg_scancount_per_dept,weekend
0,5,999,Friday,1,-1.000000,-1,1,1,-1.000000,-1.000000,0
1,7,30,Friday,2,1.000000,2,2,2,1.000000,1.000000,0
2,8,26,Friday,20,1.217391,28,6,16,1.400000,4.666667,0
3,9,8,Friday,3,1.000000,3,2,3,1.000000,1.500000,0
4,10,8,Friday,3,1.000000,3,2,3,1.000000,1.500000,0
...,...,...,...,...,...,...,...,...,...,...,...
95669,191343,25,Sunday,7,1.285714,9,3,5,1.285714,3.000000,1
95670,191344,22,Sunday,5,1.000000,5,2,3,1.000000,2.500000,1
95671,191345,39,Sunday,13,1.307692,17,8,12,1.307692,2.125000,1
95672,191346,39,Sunday,17,1.000000,17,8,16,1.000000,2.125000,1
